In [1]:
import numpy as np

from feature_treatment import load_images_from_folder, feature_extraction_set, feature_matching_set
from utils import connect_images, fill_pts3d, plot_model
from read_write_model import rotmat2qvec, qvec2rotmat, Camera
from camera_treatment import eight_point_algorithm, essential_from_fundamental, pose_from_essential, double_disambiguation, calculate_projection_matrix
from triangulation import linear_triangulation, linear_triangulation2, nonlinear_triangulation

In [6]:
cameras = {}
punts3D = {}

In [ ]:
images, img_db = load_images_from_folder('dinos')

In [7]:
 # Feature extraction
kp, des = feature_extraction_set(images, img_db)

# Feature matching
matches = feature_matching_set(kp, des)

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\flann\src\miniflann.cpp:336: error: (-210:Unsupported format or combination of formats) in function 'cv::flann::buildIndex_'
> type=0
> 

In [ ]:
adj_matrix = connect_images(matches)

In [ ]:
# Inicializar camara
height, width = images.shape[1:3]
K = np.array([  # for dino
    [2360, 0, width / 2],
    [0, 2360, height / 2],
    [0, 0, 1]])

cameras[1] = Camera(
                id=1,
                model="OPENCV",
                width=width,
                height=height,
                params=[2360, 2360, width / 2, height / 2, 0,0,0,0],
            )
punts3D = {}

In [ ]:
# Define RT for camera 1 (center at world origin and matching orientation)
RT1 = np.hstack((np.eye(3), np.zeros((3, 1))))
img_db[1].qvec = rotmat2qvec(np.eye(3))
img_db[1].tvec = np.zeros(3)

In [ ]:
 # Fundamental matrix
pts1 = np.transpose([kp[0][m.queryIdx].pt for m in matches[0][1]])
pts2 = np.transpose([kp[1][m.trainIdx].pt for m in matches[0][1]])

F = eight_point_algorithm(pts1, pts2)

# Essential matrix
E = essential_from_fundamental(K, F, K)  # In this case, the same intrinsic values apply to all images

# Get camera extrinsics from Essential matrix
RT2s = pose_from_essential(E)

In [ ]:
pts3d = np.array([linear_triangulation(K, RT1, K, RT2, pts1, pts2) for RT2 in RT2s])

RT2, pts_cloud = double_disambiguation(K, RT1, K, RT2s, pts1, pts2, pts3d)

#pts_cloud = nonlinear_triangulation(K, RT1, RT2, pts_cloud, pts1, pts2)

img_db[2].qvec = rotmat2qvec(RT2[:, :3])
img_db[2].tvec = RT2[:, -1]

In [ ]:
fill_pts3d(img_db, punts3D, pts_cloud, kp[0], matches, images, 0, 1)

In [ ]:
plot_model(punts3D)

In [ ]:
for i in range(2, images.shape[0]):
    # Get projection matrix, rotation and translation
    pts2d = np.array([pt for pt, idx in zip(img_db[i+1].xys, img_db[i+1].point3D_idxs) if idx != -1])
    pts3d = np.array([punts3D[idx].xyz for idx in img_db[i+1].point3D_idxs if idx != -1])
    P = calculate_projection_matrix(K, pts3d, pts2d)
    RT = np.linalg.inv(K) @ P
    img_db[i+1].qvec = rotmat2qvec(RT[:, :3])
    img_db[i+1].tvec = RT[:, -1]

    # Triangulate all matches with matching images
    for j in range(adj_matrix.shape[1]):
        if adj_matrix[i][j] != 1 or not isinstance(img_db[j+1].qvec, np.ndarray):
            continue

        pts2d1_idxs, pts2d2_idxs = [], []
        for m in matches[i][j]:
            if img_db[i+1].point3D_idxs[m.queryIdx] == -1:
                pts2d1_idxs.append(m.queryIdx)
                pts2d2_idxs.append(m.trainIdx)
        
        RT1 = RT
        RT2 = np.hstack((qvec2rotmat(img_db[j+1].qvec), img_db[j+1].tvec[:, np.newaxis]))
        pts2d1 = img_db[i+1].xys[pts2d1_idxs].T
        pts2d2 = img_db[j+1].xys[pts2d2_idxs].T

        pts_cloud = linear_triangulation(K, RT1, K, RT2, pts2d1, pts2d2)

        fill_pts3d(img_db, punts3D, pts_cloud, kp[i], matches, images, i, j)

In [ ]:
# for i in range(2, len(img_db)):
#     # Get projection matrix, rotation and translation
#     pts2d = np.array([pt for pt, idx in zip(img_db[i+1].xys, img_db[i+1].point3D_idxs) if idx != -1])
#     pts3d = np.array([punts3D[idx].xyz for idx in img_db[i+1].point3D_idxs if idx != -1])
#     P = calculate_projection_matrix(K, pts3d, pts2d)
#     RT = np.linalg.inv(K) @ P
#     img_db[i+1].qvec = rotmat2qvec(RT[:, :3])
#     img_db[i+1].tvec = RT[:, -1]
    
#     # Triangulate all matches with previous images and
#     for j in range(i):
#         if len(matches[(j,i)]) == 0:
#             continue
        
#         pts2d1_idxs = np.array([m.queryIdx for m in matches[(j,i)] if img_db[i+1].point3D_idxs[m.trainIdx] == -1]).astype(int)
#         pts2d2_idxs = np.array([m.trainIdx for m in matches[(j,i)] if img_db[i+1].point3D_idxs[m.trainIdx] == -1]).astype(int)

#         RTj = np.hstack((qvec2rotmat(img_db[j+1].qvec), img_db[j+1].tvec[:, np.newaxis]))
#         RTi = RT
        
#         pts_cloud = linear_triangulation(K, RTj, K, RT, img_db[j+1].xys[pts2d1_idxs, :].T, img_db[i+1].xys[pts2d2_idxs, :].T)

#         fill_pts3d(img_db, punts3D, pts_cloud, kp[j], matches, images, j, i)


In [ ]:
plot_model(punts3D)